In [1]:
import pandas as pd
from collections import defaultdict
import  time

In [2]:
df = pd.read_csv('../../../../Recherche/flexique/git/distrib/nlexique.csv')
df = df[df.gen!='b']
df[['lexeme','gen','sg']]

,lexeme,gen,sg
0,à-côté,m,akOte
1,à-coup,m,aku
2,à-peu-près,m,apØpʁɛ
3,à-pic,m,apik
4,à-plat,m,apla
...,...,...,...
31000,zurichois,m,zyʁikwa
31001,zydeco,f,zidəko
31002,zygoma,m,zigOma
31003,zygote,m,zigɔt


In [3]:
# Load the necessary packages and define complementary functions

import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib.pyplot as plt

from gensim.models import KeyedVectors



Reading frequency data for the corpus that was used for the vectors. This is useful to pick words that are frequent enough that we have quality vectors.

In [4]:
freq = pd.read_csv('../../../../Recherche/corpus/frcowvec/frequencies-frcowvec.csv')
freq = freq[freq.word.str.endswith('NOM')]
freq.index = freq.word.str.split('_').str[0]
freq=freq[freq.freq>=100]
dd = defaultdict(int)
freq = (freq.freq).to_dict(into=dd)

In [5]:
df['freq'] = df.lexeme.apply(lambda k:freq[k])
df=df[df.freq>=100]
lemmas = df.lexeme.to_list()
df

,lexeme,variants,gen,sg,pl,freq
0,à-côté,à-côté:accoté,m,akOte,akOte,1583
1,à-coup,à-coup,m,aku,aku,829
2,à-peu-près,à-peu-près,m,apØpʁɛ,apØpʁɛ,1184
3,à-pic,à-pic,m,apik,apik,241
4,à-plat,à-plat:aplat,m,apla,apla,314
...,...,...,...,...,...,...
30997,zozo,zozo,m,zozo,zozo,5929
30998,zozotement,zozotement,m,zOzɔtəmɑ̃,zOzɔtəmɑ̃,121
31001,zydeco,zydeco,f,zidəko,zidəko,181
31003,zygote,zygote,m,zigɔt,zigɔt,837


Loading vector space

In [6]:
vectors = {}
vecspace = open('/Users/olivierbonami/Dropbox/Fichiers/Recherche/corpus/frcowvec/lemma-A-pos-bow.txt','r')
for line in vecspace:
    items = line.rstrip().split(' ')
    lemma = items[0]
    vector = [float(s) for s in items[1:]]
    if lemma[-4:]=='_nom' and lemma[:-4] in lemmas:
        vectors[lemma[:-4]] = vector


Creating a Series `gender` containing the genders of all nouns in random order, and then a DataFrame `dis_pred` with the corresponding vectors.

In [7]:
df.index=df.lexeme
genders = df.gen.sample(frac=1)
print(len(df.gen[df.gen=='m'])/len(df.gen))

dis_pred = pd.DataFrame(vectors).T.loc[genders.index,:]




0.57676417121137


Before creating the phonologicla predictors, we need to normalize the transcriptions, because a few sounds are coded on two characters.

In [8]:
def normalize(s):
    s=s.replace('œ̃','1')
    s=s.replace('ɛ̃','5')
    s=s.replace('ɑ̃','3')
    return s.replace('ɔ̃','4')

df['phon']=df.sg.apply(normalize)


Now creating a DataFrame `phon_pred`. This is the concatenation of one-hot encodings of the last three characters.

In [9]:
one = pd.get_dummies(df.phon.str[-1].loc[genders.index]).astype(int)
one.columns=['-1:'+i for i in one.columns]
two = pd.get_dummies(df.phon.str[-2].loc[genders.index]).astype(int)
two.columns=['-2:'+i for i in two.columns]
three = pd.get_dummies(df.phon.str[-3].loc[genders.index]).astype(int)
three.columns=['-3:'+i for i in three.columns]
phon_pred=pd.concat([one,two,three],axis=1)
phon_pred

,-1:#,-1:1,-1:3,-1:4,-1:5,-1:E,-1:a,-1:b,-1:d,-1:e,...,-3:ŋ,-3:œ,-3:ɔ,-3:ə,-3:ɛ,-3:ɥ,-3:ɲ,-3:ʁ,-3:ʃ,-3:ʒ
lexeme,,,,,,,,,,,,,,,,,,,,,
accident,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
empiétement,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
glabelle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
lancer,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
regardeur,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
latence,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
capitanat,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aînée,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Now we can start training models. Hyperparameters have been optimized by grid search, I'm skipping the code for that.

Note that using such a high number of estimators (500) leads to very long computation times. I really tried to see how far I could push this. I had reasonably good numbers with just 60 estimators, I recommend using that for exploratory purposes. 

Phonological prediction (takes 370s on my machine):

In [10]:
n=500
lr=.1
max_depth=15
min_samples_split=10
start = time.time()
BT = GradientBoostingClassifier(n_estimators=n, 
                          learning_rate=lr, 
                          max_depth=max_depth, 
                          min_samples_split = min_samples_split,
                          random_state=0,
                          loss='log_loss')
phon_pred_res = cross_val_predict(BT, phon_pred, genders, cv=10)
acc = round(sum(phon_pred_res==genders)/len(genders),3)
duration = int(round(time.time() -start,0))
print(acc, ' (',duration,'s)',sep='')

0.848 (366s)


Prediction from embeddings **beware, this takes about 7000s on my machine**

In [11]:
start = time.time()
BT = GradientBoostingClassifier(n_estimators=n, 
                          learning_rate=lr, 
                          max_depth=max_depth, 
                          min_samples_split = min_samples_split,
                          random_state=0,
                          loss='log_loss')
dis_pred_res = cross_val_predict(BT, dis_pred, genders, cv=10)
acc = round(sum(dis_pred_res==genders)/len(genders),3)
duration = int(round(time.time() -start,0))
print(acc, ' (',duration,'s)',sep='')

0.735 (6832s)


Prediction from both phonology and embeddings **beware, this takes about 18000s on my machine**

In [12]:
both = pd.concat([dis_pred,phon_pred],axis=1)
both.columns=both.columns.astype(str)

start = time.time()
BT = GradientBoostingClassifier(n_estimators=n, 
                          learning_rate=lr, 
                          max_depth=max_depth, 
                          min_samples_split = min_samples_split,
                          random_state=0,
                          loss='log_loss')
both_res = cross_val_predict(BT, both, genders, cv=10)
acc = round(sum(both_res==genders)/len(genders),3)
duration = int(round(time.time() -start,0))
print(acc, ' (',duration,'s)',sep='')

0.888 (18111s)
